In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [53]:
mol_name = 'H2O'
# mol_name = 'H2'

In [54]:
xyz_file = xyz_from_pubchem(mol_name)

In [4]:
# # xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'
# # print(xyz_file)


# xyz_file = '4\n \nH\t0\t0\t0\nH\t2.45366053071732\t0\t0\nH\t2.45366053071732\t2.45366053071732\t0\nH\t0\t2.45366053071732\t0'
# # geometry=[
# #     ('H', (0.0,0.0,0.0)),
# #     ('H', (2.45366053071732,0.0,0.0)),
# #     ('H', (2.45366053071732,2.45366053071732,0.0)),
# #     ('H', (0.0,2.45366053071732,0.0))
# #      ]  
# print(xyz_file)

In [55]:
from symred.chem import PySCFDriver

In [56]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=1000
ram = 8_000
run_mp2  = False,
run_cisd = False,
run_ccsd = True,
run_fci  = False

In [57]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [58]:
pyscf_obj.run_pyscf()

In [59]:
2*pyscf_obj.pyscf_hf.mol.nao

14

In [60]:
from symred.chem import FermionicHamilt

In [61]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 9.08436451197603
((0, 1), (0, 0)) -32.68895210663272
((0, 1), (2, 0)) 0.5582454628694448
((0, 1), (4, 0)) 2.54022894090807e-07
((0, 1), (6, 0)) 0.2660800253847062
((0, 1), (10, 0)) 0.31340928606888
((0, 1), (12, 0)) -1.477797333210057e-06
((1, 1), (1, 0)) -32.68895210663272
((1, 1), (3, 0)) 0.5582454628694448
((1, 1), (5, 0)) 2.54022894090807e-07
((1, 1), (7, 0)) 0.2660800253847062
((1, 1), (11, 0)) 0.31340928606888
((1, 1), (13, 0)) -1.477797333210057e-06
((2, 1), (0, 0)) 0.5582454628694435
((2, 1), (2, 0)) -7.581925631735151
((2, 1), (4, 0)) 3.787547667638218e-08
((2, 1), (6, 0)) -0.488495142106376
((2, 1), (10, 0)) -1.4460616540254612
((2, 1), (12, 0)) 7.007873807317976e-06
((3, 1), (1, 0)) 0.5582454628694435
((3, 1), (3, 0)) -7.581925631735151
((3, 1), (5, 0)) 3.787547667638218e-08
((3, 1), (7, 0)) -0.488495142106376
((3, 1), (11, 0)) -1.4460616540254612
((3, 1), (13, 0)) 7.007873807317976e-06
((4, 1), (0, 0)) 2.5402289396460144e-07
((4, 1), (2, 0)) 3.787547678669034e-08
((4, 1)

In [62]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [63]:
hf_ket = H_ferm.hf_ket
H_mat = H_ferm.get_sparse_ham()

In [64]:
from pyscf.cc.addons import spatial2spin

# doubles!

t2 = spatial2spin(pyscf_obj.pyscf_ccsd.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [65]:
# singles

t1 = spatial2spin(pyscf_obj.pyscf_ccsd.t1)
no, nv = t1.shape
nmo = no + nv
ccsd_single_amps = np.zeros((nmo, nmo))
ccsd_single_amps[no:,:no] = t1.T
ccsd_single_amps.shape

(14, 14)

In [66]:
double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[10, 0, 11, 1], -0.00032052932827647784],
 [[10, 0, 11, 3], -0.00015763954303633237],
 [[10, 0, 11, 7], 5.390573819177737e-05],
 [[10, 0, 12, 4], 0.00023666272053453143],
 [[10, 0, 13, 5], 0.0001171283735429166],
 [[10, 1, 11, 0], 0.00032052932827647784],
 [[10, 1, 11, 2], 0.00015763954303633237],
 [[10, 1, 11, 6], -5.390573819177737e-05],
 [[10, 1, 13, 4], 0.00011953434699161483],
 [[10, 2, 11, 1], -0.00015763954303633237],
 [[10, 2, 11, 3], -0.01771271459964041],
 [[10, 2, 11, 5], -1.2839455344458697e-07],
 [[10, 2, 11, 7], 0.017112925632578137],
 [[10, 2, 12, 4], -0.005623520381703141],
 [[10, 2, 13, 3], 4.5400146715495126e-08],
 [[10, 2, 13, 5], -0.016805083557423856],
 [[10, 2, 13, 7], -7.506773862479564e-08],
 [[10, 3, 11, 0], 0.00015763954303633237],
 [[10, 3, 11, 2], 0.01771271459964041],
 [[10, 3, 11, 4], 1.2839455344458697e-07],
 [[10, 3, 11, 6], -0.017112925632578137],
 [[10, 3, 13, 2], -4.5400146715495126e-08],
 [[10, 3, 13, 4], 0.011181563175720714],
 [[10, 3, 13, 6], 7.

In [17]:
single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

single_amplitudes_list

[[[2, 0], 1.1022680481285446e-16], [[3, 1], 1.1022680481285446e-16]]

In [67]:
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [68]:
generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)

In [20]:
T = generator_t1 + 0.5*generator_t2

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))

In [21]:
ccsd_ket =  ccsd_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ ccsd_ket

(-1.1011503302444787+0j)

In [34]:
pyscf_obj.pyscf_ccsd.e_tot 

-1.1011503302444787

In [42]:
# intermediate normalization
hf_ket.conj().T  @ ccsd_ket


(1+0j)

In [28]:
normalized_state = ccsd_ket / np.sqrt(ccsd_ket.conj().T@ccsd_ket)

normalized_state.conj().T @ H_mat @ normalized_state

(-1.101150330232619+0j)

In [35]:
for ind, amp in enumerate(normalized_state):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

(-0.1753082422875078+0j)|0011>
(0.9845135957344949+0j)|1100>


In [69]:
trotter_op = np.eye(2**H_ferm.n_qubits)
for op in list(generator_t2):
    trotter_op*= expm(get_sparse_operator(op, n_qubits=H_ferm.n_qubits))

    
ccsd_trotter = trotter_op @ hf_ket


normalized_trotter = ccsd_trotter / np.sqrt(ccsd_trotter.conj().T@ccsd_trotter)

normalized_trotter.conj().T @ H_mat @ normalized_trotter

KeyboardInterrupt: 

In [52]:
for ind, amp in enumerate(normalized_trotter):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

(-0.33549144932116853+0j)|0011>
(0.9420432513597141+0j)|1100>
